In [16]:
#!conda install -y numpy matplotlib pandas ipympl scikit-learn    # not needed if running in nix shell or docker with docker.io/cschranz/gpu-jupyter:v1.9_cuda-12.6_ubuntu-24.04_python-only
!pip install numpy matplotlib pandas ipympl scikit-learn

In [1]:
# use cupy instead of numpy, for GPU offload
#!conda install -y -c conda-forge cupy cutensor nccl
!pip install cupy-cuda12x cutensor-cu12 nvidia-nccl-cu12


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 MB 90.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 92.9 MB/s eta 0:00:00a 0:00:01


# Whole workflow

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import cupy as cp
import pickle

In [3]:
hd = pd.read_csv('./heart-disease.csv')

In [4]:
# Features matrix (x)
x = hd.drop('target', axis=1)

# Labels matrix (y)
y = hd['target']

## Alogrithm selection

In [5]:
clf = RandomForestClassifier() # keeping the default hyperparameters
clf.get_params()


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [6]:
# Fitting the model to the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)  # 20% will be used for testing while 80% for training

In [7]:
clf.fit(x_train, y_train);

In [8]:
# make a prediction
y_preds = clf.predict(x_test)
print(f'Preds:\n{y_preds}\n\n')
print(f'Test:\n{y_test}')

Preds:
[1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1
 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 1]


Test:
92     1
280    0
0      1
3      1
206    0
      ..
113    1
41     1
122    1
176    0
293    0
Name: target, Length: 61, dtype: int64


In [9]:
# Evaluate the model on training data
clf.score(x_train, y_train)

1.0

In [10]:
# Evaluate the model on test data
clf.score(x_test, y_test)

0.8688524590163934

In [11]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88        33
           1       0.86      0.86      0.86        28

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61



In [12]:
confusion_matrix(y_test, y_preds)

array([[29,  4],
       [ 4, 24]])

In [13]:
accuracy_score(y_test, y_preds)

0.8688524590163934

### Improve the model

In [14]:
# Change the n_estimators hyperparam
cp.random.seed(0)

for i in range(10, 151, 10):
    print(f'Trying model with {i} estimators...')
    
    clf = RandomForestClassifier(n_estimators=i)
    clf.fit(x_train, y_train);
    print(f'Model accuracy on test set: {clf.score(x_test, y_test) * 100:.2f}')
    print()
    

Trying model with 10 estimators...
Model accuracy on test set: 78.69

Trying model with 20 estimators...
Model accuracy on test set: 77.05

Trying model with 30 estimators...
Model accuracy on test set: 80.33

Trying model with 40 estimators...
Model accuracy on test set: 81.97

Trying model with 50 estimators...
Model accuracy on test set: 83.61

Trying model with 60 estimators...
Model accuracy on test set: 78.69

Trying model with 70 estimators...
Model accuracy on test set: 80.33

Trying model with 80 estimators...
Model accuracy on test set: 83.61

Trying model with 90 estimators...
Model accuracy on test set: 80.33

Trying model with 100 estimators...
Model accuracy on test set: 85.25

Trying model with 110 estimators...
Model accuracy on test set: 81.97

Trying model with 120 estimators...
Model accuracy on test set: 83.61

Trying model with 130 estimators...
Model accuracy on test set: 81.97

Trying model with 140 estimators...
Model accuracy on test set: 83.61

Trying model wi

In [15]:
# Save the model
pickle.dump(clf, open('./random_forest_model_1.pkl', 'wb'))

# Load the model
loaded_model = pickle.load(open('./random_forest_model_1.pkl', 'rb'))
loaded_model.score(x_test, y_test)

0.8360655737704918

# Data preprocessing
Clean => Transform => Reduce
1. Spliting into features (`X`) and labels (`y`)
2. Handling missing data
3. Feature encoding (converting non-numerical values to numerical ones)

In [16]:
hd = pd.read_csv('./heart-disease.csv')
print(f'Dataset shape: {hd.shape}')

Dataset shape: (303, 14)


## Splitting data

In [17]:
X = hd.drop('target', axis=1)

In [18]:
y = hd['target']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (242, 13)
X_test shape: (61, 13)
y_train shape: (242,)
y_test shape: (61,)


## Handling missing data